In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import json
import os
import glob
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
df_transactions = pd.read_csv('/kaggle/input/transactions-fraud-datasets/transactions_data.csv')
df_transactions.head(5)

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN


In [3]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13305915 entries, 0 to 13305914
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   id              int64  
 1   date            object 
 2   client_id       int64  
 3   card_id         int64  
 4   amount          object 
 5   use_chip        object 
 6   merchant_id     int64  
 7   merchant_city   object 
 8   merchant_state  object 
 9   zip             float64
 10  mcc             int64  
 11  errors          object 
dtypes: float64(1), int64(5), object(6)
memory usage: 1.2+ GB


In [4]:
df_transactions.isna().sum()

id                       0
date                     0
client_id                0
card_id                  0
amount                   0
use_chip                 0
merchant_id              0
merchant_city            0
merchant_state     1563700
zip                1652706
mcc                      0
errors            13094522
dtype: int64

In [5]:
df_transactions.duplicated().sum()

0

In [6]:
df_transactions['id'].duplicated().sum()

0

In [3]:
df_transactions['date'] = pd.to_datetime(df_transactions['date'],errors='coerce')
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13305915 entries, 0 to 13305914
Data columns (total 12 columns):
 #   Column          Dtype         
---  ------          -----         
 0   id              int64         
 1   date            datetime64[ns]
 2   client_id       int64         
 3   card_id         int64         
 4   amount          object        
 5   use_chip        object        
 6   merchant_id     int64         
 7   merchant_city   object        
 8   merchant_state  object        
 9   zip             float64       
 10  mcc             int64         
 11  errors          object        
dtypes: datetime64[ns](1), float64(1), int64(5), object(5)
memory usage: 1.2+ GB


In [4]:
df_transactions['hour'] = df_transactions['date'].dt.hour
df_transactions.head(5)

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,hour
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN,0
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN,0
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN,0
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN,0
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN,0


In [7]:
df_transactions['date'].dt.date.min()

datetime.date(2010, 1, 1)

In [8]:
df_transactions['date'].dt.date.max()

datetime.date(2019, 10, 31)

In [5]:
df_transactions['amount'] = pd.to_numeric(
    df_transactions['amount'].astype(str).replace('[\$,]', '', regex=True),
    errors='coerce'
)
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13305915 entries, 0 to 13305914
Data columns (total 13 columns):
 #   Column          Dtype         
---  ------          -----         
 0   id              int64         
 1   date            datetime64[ns]
 2   client_id       int64         
 3   card_id         int64         
 4   amount          float64       
 5   use_chip        object        
 6   merchant_id     int64         
 7   merchant_city   object        
 8   merchant_state  object        
 9   zip             float64       
 10  mcc             int64         
 11  errors          object        
 12  hour            int32         
dtypes: datetime64[ns](1), float64(2), int32(1), int64(5), object(4)
memory usage: 1.2+ GB


In [10]:
df_transactions.loc[df_transactions['amount'] <0,['use_chip']].value_counts()

use_chip          
Swipe Transaction     389176
Chip Transaction      264190
Online Transaction      6683
Name: count, dtype: int64

In [6]:
df_transactions['transaction_type'] = np.where(df_transactions['amount'] < 0, 'Refund','Purchase')
df_transactions.head(5)

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,hour,transaction_type
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN,0,Refund
1,7475328,2010-01-01 00:02:00,561,4575,14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN,0,Purchase
2,7475329,2010-01-01 00:02:00,1129,102,80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN,0,Purchase
3,7475331,2010-01-01 00:05:00,430,2860,200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN,0,Purchase
4,7475332,2010-01-01 00:06:00,848,3915,46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN,0,Purchase


In [7]:
df_transactions['use_chip'] = df_transactions['use_chip'].str.strip().replace({
    'Swipe Transaction': 'Swipe',
    'Online Transaction': 'Online',
    'Chip Transaction': 'Chip'
})
df_transactions.head(5)

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,hour,transaction_type
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,Swipe,59935,Beulah,ND,58523.0,5499,NaN,0,Refund
1,7475328,2010-01-01 00:02:00,561,4575,14.57,Swipe,67570,Bettendorf,IA,52722.0,5311,NaN,0,Purchase
2,7475329,2010-01-01 00:02:00,1129,102,80.00,Swipe,27092,Vista,CA,92084.0,4829,NaN,0,Purchase
3,7475331,2010-01-01 00:05:00,430,2860,200.00,Swipe,27092,Crown Point,IN,46307.0,4829,NaN,0,Purchase
4,7475332,2010-01-01 00:06:00,848,3915,46.41,Swipe,13051,Harwood,MD,20776.0,5813,NaN,0,Purchase


In [13]:
df_transactions.loc[(df_transactions['merchant_state'].isna()),['merchant_city']].value_counts()

merchant_city
ONLINE           1563700
Name: count, dtype: int64

In [14]:
df_transactions.loc[(df_transactions['merchant_state'].isna()) & (df_transactions['use_chip'] == 'Online'),['merchant_city']].value_counts()

merchant_city
ONLINE           1557912
Name: count, dtype: int64

In [15]:
df_transactions.loc[(df_transactions['merchant_state'].isna()) & (df_transactions['use_chip'] == 'Chip'),['merchant_city']].value_counts()

merchant_city
ONLINE           5788
Name: count, dtype: int64

In [8]:
df_transactions["merchant_state"].fillna('ONLINE', inplace= True)
df_transactions.isna().sum()

/tmp/ipykernel_48/790516748.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_transactions["merchant_state"].fillna('ONLINE', inplace= True)


id                         0
date                       0
client_id                  0
card_id                    0
amount                     0
use_chip                   0
merchant_id                0
merchant_city              0
merchant_state             0
zip                  1652706
mcc                        0
errors              13094522
hour                       0
transaction_type           0
dtype: int64

In [17]:
df_transactions.loc[(df_transactions['zip'].isna()),['use_chip']].value_counts()

use_chip
Online      1557912
Swipe         48493
Chip          46301
Name: count, dtype: int64

In [18]:
df_transactions.loc[(df_transactions['zip'].isna()) & (df_transactions['use_chip'] == 'Swipe'),['merchant_city']].value_counts()

merchant_city      
Cancun                 5792
Mexico City            2960
Puerto Vallarta        2699
Cabo San Lucas         2630
Toronto                2058
Rome                   1730
London                 1720
Berlin                 1711
Guadalajara            1603
Paris                  1572
Montreal               1488
Tokyo                  1310
Beijing                1076
Vancouver               958
Santo Domingo           879
Amsterdam               821
Edinburgh               810
Seoul                   705
Jerusalem               692
Calgary                 628
Nassau                  626
Kingston                605
Acapulco                578
Tapei                   559
Bogota                  507
Shanghai                495
Madrid                  470
Dublin                  466
Barcelona               461
Zurich                  420
Manila                  410
Vienna                  393
Port au Prince          383
Oranjestad              376
Geneva                  332


In [19]:
df_transactions.loc[(df_transactions['zip'].isna()) & (df_transactions['use_chip'] == 'Chip'),['merchant_city']].value_counts()

merchant_city   
ONLINE              5788
Rome                5351
Cancun              3533
Toronto             2239
Mexico City         2142
Cabo San Lucas      1920
Puerto Vallarta     1743
London              1401
Guadalajara         1357
Berlin              1314
Paris               1264
Vancouver           1186
Beijing              857
Tokyo                797
Montreal             778
Santo Domingo        680
Amsterdam            677
Edmonton             578
Kingston             571
Edinburgh            551
San Jose             492
Dublin               492
Shanghai             480
Seoul                448
Acapulco             444
Madrid               410
Calgary              409
Tapei                403
Nassau               369
Bogota               355
Barcelona            310
Hong Kong            309
Delhi                307
Sydney               293
Manila               254
Jerusalem            249
Mumbai               247
Lima                 244
Vienna               220
Lisbon  

In [9]:
df_transactions['zip'].fillna('00000', inplace= True)
df_transactions.isna().sum()

/tmp/ipykernel_48/23149537.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_transactions['zip'].fillna('00000', inplace= True)
/tmp/ipykernel_48/23149537.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '00000' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_transactions['zip'].fillna('00000', inplace= True)


id                         0
date                       0
client_id                  0
card_id                    0
amount                     0
use_chip                   0
merchant_id                0
merchant_city              0
merchant_state             0
zip                        0
mcc                        0
errors              13094522
hour                       0
transaction_type           0
dtype: int64

In [99]:
df_transactions.loc[df_transactions['errors'].notna(),['errors']].value_counts()

errors                                             
Insufficient Balance                                   130902
Bad PIN                                                 32119
Technical Glitch                                        26271
Bad Card Number                                          7767
Bad Expiration                                           6161
Bad CVV                                                  6106
Bad Zipcode                                              1126
Bad PIN,Insufficient Balance                              293
Insufficient Balance,Technical Glitch                     243
Bad Card Number,Insufficient Balance                       71
Bad PIN,Technical Glitch                                   70
Bad CVV,Insufficient Balance                               57
Bad Expiration,Insufficient Balance                        47
Bad Card Number,Bad CVV                                    38
Bad Card Number,Bad Expiration                             33
Bad Expiration,Bad

In [10]:
df_transactions['errors'].fillna('No Errors', inplace = True)
df_transactions.isna().sum()

/tmp/ipykernel_48/1104083274.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_transactions['errors'].fillna('No Errors', inplace = True)


id                  0
date                0
client_id           0
card_id             0
amount              0
use_chip            0
merchant_id         0
merchant_city       0
merchant_state      0
zip                 0
mcc                 0
errors              0
hour                0
transaction_type    0
dtype: int64

In [23]:
df_transactions.head(5)

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,hour,transaction_type
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,Swipe,59935,Beulah,ND,58523.0,5499,No Errors,0,Refund
1,7475328,2010-01-01 00:02:00,561,4575,14.57,Swipe,67570,Bettendorf,IA,52722.0,5311,No Errors,0,Purchase
2,7475329,2010-01-01 00:02:00,1129,102,80.00,Swipe,27092,Vista,CA,92084.0,4829,No Errors,0,Purchase
3,7475331,2010-01-01 00:05:00,430,2860,200.00,Swipe,27092,Crown Point,IN,46307.0,4829,No Errors,0,Purchase
4,7475332,2010-01-01 00:06:00,848,3915,46.41,Swipe,13051,Harwood,MD,20776.0,5813,No Errors,0,Purchase


In [24]:
df_transactions['merchant_id'].nunique()

74831

In [25]:
df_transactions.groupby('merchant_id')['merchant_city']\
    .nunique()\
    .sort_values(ascending=False)

merchant_id
59935     2579
61195     1996
43293     1979
75936     1776
14528     1741
          ... 
37460        1
37461        1
37462        1
37463        1
100342       1
Name: merchant_city, Length: 74831, dtype: int64

In [11]:
df_transactions['merchant_key'] = np.where(df_transactions['merchant_city'].str.upper() == 'ONLINE',\
    'ONLINE',\
    df_transactions['merchant_id'].astype(str) + '_' + df_transactions['merchant_city'])
df_transactions.head(10)


,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,hour,transaction_type,merchant_key
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,Swipe,59935,Beulah,ND,58523.0,5499,No Errors,0,Refund,59935_Beulah
1,7475328,2010-01-01 00:02:00,561,4575,14.57,Swipe,67570,Bettendorf,IA,52722.0,5311,No Errors,0,Purchase,67570_Bettendorf
2,7475329,2010-01-01 00:02:00,1129,102,80.00,Swipe,27092,Vista,CA,92084.0,4829,No Errors,0,Purchase,27092_Vista
3,7475331,2010-01-01 00:05:00,430,2860,200.00,Swipe,27092,Crown Point,IN,46307.0,4829,No Errors,0,Purchase,27092_Crown Point
4,7475332,2010-01-01 00:06:00,848,3915,46.41,Swipe,13051,Harwood,MD,20776.0,5813,No Errors,0,Purchase,13051_Harwood
5,7475333,2010-01-01 00:07:00,1807,165,4.81,Swipe,20519,Bronx,NY,10464.0,5942,No Errors,0,Purchase,20519_Bronx
6,7475334,2010-01-01 00:09:00,1556,2972,77.00,Swipe,59935,Beulah,ND,58523.0,5499,No Errors,0,Purchase,59935_Beulah
7,7475335,2010-01-01 00:14:00,1684,2140,26.46,Online,39021,ONLINE,ONLINE,00000,4784,No Errors,0,Purchase,ONLINE
8,7475336,2010-01-01 00:21:00,335,5131,261.58,Online,50292,ONLINE,ONLINE,00000,7801,No Errors,0,Purchase,ONLINE
9,7475337,2010-01-01 00:21:00,351,1112,10.74,Swipe,3864,Flushing,NY,11355.0,5813,No Errors,0,Purchase,3864_Flushing


In [12]:
df_transactions['merchant_key'] = np.where(df_transactions['merchant_key'].str.upper() == 'ONLINE',\
    df_transactions['merchant_key'].astype(str) + '_' + df_transactions['merchant_id'].astype(str),\
    df_transactions['merchant_key'])
df_transactions.head(5)


,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,hour,transaction_type,merchant_key
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,Swipe,59935,Beulah,ND,58523.0,5499,No Errors,0,Refund,59935_Beulah
1,7475328,2010-01-01 00:02:00,561,4575,14.57,Swipe,67570,Bettendorf,IA,52722.0,5311,No Errors,0,Purchase,67570_Bettendorf
2,7475329,2010-01-01 00:02:00,1129,102,80.00,Swipe,27092,Vista,CA,92084.0,4829,No Errors,0,Purchase,27092_Vista
3,7475331,2010-01-01 00:05:00,430,2860,200.00,Swipe,27092,Crown Point,IN,46307.0,4829,No Errors,0,Purchase,27092_Crown Point
4,7475332,2010-01-01 00:06:00,848,3915,46.41,Swipe,13051,Harwood,MD,20776.0,5813,No Errors,0,Purchase,13051_Harwood


# **FRAUD LABELS**

In [13]:
fraud_labels = pd.read_json("/kaggle/input/transactions-fraud-datasets/train_fraud_labels.json")
fraud_labels.reset_index(inplace= True)
fraud_labels.columns = ["transaction_id","is_fraud"]
fraud_labels.head(10)

,transaction_id,is_fraud
0,10649266,No
1,23410063,No
2,9316588,No
3,12478022,No
4,9558530,No
5,12532830,No
6,19526714,No
7,9906964,No
8,13224888,No
9,13749094,No


In [29]:
fraud_labels['transaction_id'].duplicated().sum()

0

In [30]:
fraud_labels['is_fraud'].unique()

array(['No', 'Yes'], dtype=object)

In [31]:
print(f"Total Rows of Fraud Labels: {fraud_labels['transaction_id'].nunique()}")
print(f"Total Rows of Fraud Transactions: {df_transactions['id'].nunique()}")

Total Rows of Fraud Labels: 8914963
Total Rows of Fraud Transactions: 13305915


In [14]:
df_transactions = df_transactions.merge(fraud_labels,left_on='id',right_on='transaction_id',how='left').reset_index(drop=True)
df_transactions.head(5)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,hour,transaction_type,merchant_key,transaction_id,is_fraud
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,Swipe,59935,Beulah,ND,58523.0,5499,No Errors,0,Refund,59935_Beulah,7475327.0,No
1,7475328,2010-01-01 00:02:00,561,4575,14.57,Swipe,67570,Bettendorf,IA,52722.0,5311,No Errors,0,Purchase,67570_Bettendorf,7475328.0,No
2,7475329,2010-01-01 00:02:00,1129,102,80.00,Swipe,27092,Vista,CA,92084.0,4829,No Errors,0,Purchase,27092_Vista,7475329.0,No
3,7475331,2010-01-01 00:05:00,430,2860,200.00,Swipe,27092,Crown Point,IN,46307.0,4829,No Errors,0,Purchase,27092_Crown Point,NaN,NaN
4,7475332,2010-01-01 00:06:00,848,3915,46.41,Swipe,13051,Harwood,MD,20776.0,5813,No Errors,0,Purchase,13051_Harwood,7475332.0,No


In [76]:
df_transactions.columns.tolist()

['id',
 'date',
 'client_id',
 'card_id',
 'amount',
 'use_chip',
 'merchant_id',
 'merchant_city',
 'merchant_state',
 'zip',
 'mcc',
 'errors',
 'hour',
 'transaction_type',
 'merchant_key',
 'transaction_id',
 'is_fraud']

In [34]:
df_transactions.isna().sum()

id                        0
date                      0
client_id                 0
card_id                   0
amount                    0
use_chip                  0
merchant_id               0
merchant_city             0
merchant_state            0
zip                       0
mcc                       0
errors                    0
hour                      0
transaction_type          0
merchant_key              0
transaction_id      4390952
is_fraud            4390952
dtype: int64

In [15]:
df_transactions.drop('transaction_id',axis=1,inplace=True)
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13305915 entries, 0 to 13305914
Data columns (total 16 columns):
 #   Column            Dtype         
---  ------            -----         
 0   id                int64         
 1   date              datetime64[ns]
 2   client_id         int64         
 3   card_id           int64         
 4   amount            float64       
 5   use_chip          object        
 6   merchant_id       int64         
 7   merchant_city     object        
 8   merchant_state    object        
 9   zip               object        
 10  mcc               int64         
 11  errors            object        
 12  hour              int32         
 13  transaction_type  object        
 14  merchant_key      object        
 15  is_fraud          object        
dtypes: datetime64[ns](1), float64(1), int32(1), int64(5), object(8)
memory usage: 1.5+ GB


In [16]:
df_transactions['is_fraud'].fillna('Inspect', inplace = True)
df_transactions.isna().sum()

/tmp/ipykernel_48/3109392893.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_transactions['is_fraud'].fillna('Inspect', inplace = True)


id                  0
date                0
client_id           0
card_id             0
amount              0
use_chip            0
merchant_id         0
merchant_city       0
merchant_state      0
zip                 0
mcc                 0
errors              0
hour                0
transaction_type    0
merchant_key        0
is_fraud            0
dtype: int64

In [79]:
df_transactions.groupby('merchant_key')['merchant_city']\
    .nunique()\
    .sort_values(ascending=False)

merchant_key
100001_South Park         1
69118_Clifton             1
69111_Austin              1
69112_Allendale           1
69113_Crawford            1
                         ..
38489_Muskegon            1
38489_Neshanic Station    1
38489_New Albany          1
38489_New York            1
ONLINE_99370              1
Name: merchant_city, Length: 192902, dtype: int64

In [80]:
df_transactions['merchant_key'].nunique()

192902

# **MERCHANT**

In [81]:
df_merchant = df_transactions[['merchant_key','merchant_id','merchant_city','merchant_state','zip','mcc']]
df_merchant.head(5)

,merchant_key,merchant_id,merchant_city,merchant_state,zip,mcc
0,59935_Beulah,59935,Beulah,ND,58523.0,5499
1,67570_Bettendorf,67570,Bettendorf,IA,52722.0,5311
2,27092_Vista,27092,Vista,CA,92084.0,4829
3,27092_Crown Point,27092,Crown Point,IN,46307.0,4829
4,13051_Harwood,13051,Harwood,MD,20776.0,5813


In [82]:
df_merchant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13305915 entries, 0 to 13305914
Data columns (total 6 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   merchant_key    object
 1   merchant_id     int64 
 2   merchant_city   object
 3   merchant_state  object
 4   zip             object
 5   mcc             int64 
dtypes: int64(2), object(4)
memory usage: 609.1+ MB


In [83]:
df_merchant = df_merchant.drop_duplicates(subset = ['merchant_key']).reset_index(drop = True)
df_merchant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192902 entries, 0 to 192901
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   merchant_key    192902 non-null  object
 1   merchant_id     192902 non-null  int64 
 2   merchant_city   192902 non-null  object
 3   merchant_state  192902 non-null  object
 4   zip             192902 non-null  object
 5   mcc             192902 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 8.8+ MB


In [84]:
df_merchant['merchant_key'].duplicated().sum()

0

In [87]:
df_merchant['merchant_id'].nunique()

74831

In [107]:
df_transactions.head(5)

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,hour,transaction_type,merchant_key,is_fraud
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,Swipe,59935,Beulah,ND,58523.0,5499,No Errors,0,Refund,59935_Beulah,No
1,7475328,2010-01-01 00:02:00,561,4575,14.57,Swipe,67570,Bettendorf,IA,52722.0,5311,No Errors,0,Purchase,67570_Bettendorf,No
2,7475329,2010-01-01 00:02:00,1129,102,80.00,Swipe,27092,Vista,CA,92084.0,4829,No Errors,0,Purchase,27092_Vista,No
3,7475331,2010-01-01 00:05:00,430,2860,200.00,Swipe,27092,Crown Point,IN,46307.0,4829,No Errors,0,Purchase,27092_Crown Point,Inspect
4,7475332,2010-01-01 00:06:00,848,3915,46.41,Swipe,13051,Harwood,MD,20776.0,5813,No Errors,0,Purchase,13051_Harwood,No


In [17]:
df_transactions = df_transactions[['id','date','hour','client_id','card_id','merchant_key','amount','use_chip','transaction_type','mcc','errors','is_fraud']]
df_transactions.head(5)

,id,date,hour,client_id,card_id,merchant_key,amount,use_chip,transaction_type,mcc,errors,is_fraud
0,7475327,2010-01-01 00:01:00,0,1556,2972,59935_Beulah,-77.00,Swipe,Refund,5499,No Errors,No
1,7475328,2010-01-01 00:02:00,0,561,4575,67570_Bettendorf,14.57,Swipe,Purchase,5311,No Errors,No
2,7475329,2010-01-01 00:02:00,0,1129,102,27092_Vista,80.00,Swipe,Purchase,4829,No Errors,No
3,7475331,2010-01-01 00:05:00,0,430,2860,27092_Crown Point,200.00,Swipe,Purchase,4829,No Errors,Inspect
4,7475332,2010-01-01 00:06:00,0,848,3915,13051_Harwood,46.41,Swipe,Purchase,5813,No Errors,No


## **CARDS DATA**

In [33]:
df_cards = pd.read_csv('/kaggle/input/transactions-fraud-datasets/cards_data.csv')
df_cards.head(5)

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No


In [34]:
df_cards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6146 entries, 0 to 6145
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     6146 non-null   int64 
 1   client_id              6146 non-null   int64 
 2   card_brand             6146 non-null   object
 3   card_type              6146 non-null   object
 4   card_number            6146 non-null   int64 
 5   expires                6146 non-null   object
 6   cvv                    6146 non-null   int64 
 7   has_chip               6146 non-null   object
 8   num_cards_issued       6146 non-null   int64 
 9   credit_limit           6146 non-null   object
 10  acct_open_date         6146 non-null   object
 11  year_pin_last_changed  6146 non-null   int64 
 12  card_on_dark_web       6146 non-null   object
dtypes: int64(6), object(7)
memory usage: 624.3+ KB


In [35]:
df_cards['id'].duplicated().sum()

0

In [36]:
df_cards.isna().sum()

id                       0
client_id                0
card_brand               0
card_type                0
card_number              0
expires                  0
cvv                      0
has_chip                 0
num_cards_issued         0
credit_limit             0
acct_open_date           0
year_pin_last_changed    0
card_on_dark_web         0
dtype: int64

In [37]:
df_cards['expires'] = pd.to_datetime(df_cards['expires'], errors='coerce')
df_cards['acct_open_date'] = pd.to_datetime(df_cards['acct_open_date'], errors='coerce')
df_cards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6146 entries, 0 to 6145
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     6146 non-null   int64         
 1   client_id              6146 non-null   int64         
 2   card_brand             6146 non-null   object        
 3   card_type              6146 non-null   object        
 4   card_number            6146 non-null   int64         
 5   expires                6146 non-null   datetime64[ns]
 6   cvv                    6146 non-null   int64         
 7   has_chip               6146 non-null   object        
 8   num_cards_issued       6146 non-null   int64         
 9   credit_limit           6146 non-null   object        
 10  acct_open_date         6146 non-null   datetime64[ns]
 11  year_pin_last_changed  6146 non-null   int64         
 12  card_on_dark_web       6146 non-null   object        
dtypes: 

/tmp/ipykernel_151/556905016.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_cards['expires'] = pd.to_datetime(df_cards['expires'], errors='coerce')
/tmp/ipykernel_151/556905016.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_cards['acct_open_date'] = pd.to_datetime(df_cards['acct_open_date'], errors='coerce')


In [38]:
df_cards['card_brand'].unique()

array(['Visa', 'Mastercard', 'Discover', 'Amex'], dtype=object)

In [39]:
df_cards['card_type'].unique()

array(['Debit', 'Credit', 'Debit (Prepaid)'], dtype=object)

In [40]:
df_cards.loc[df_cards['card_type'] == 'Debit (Prepaid)','card_type'] = 'Debit'
df_cards['card_type'].unique()

array(['Debit', 'Credit'], dtype=object)

In [41]:
df_cards['credit_limit'] = pd.to_numeric(
    df_cards['credit_limit'].astype(str).replace('[\$,]', '', regex=True),
    errors='coerce'
)
df_cards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6146 entries, 0 to 6145
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     6146 non-null   int64         
 1   client_id              6146 non-null   int64         
 2   card_brand             6146 non-null   object        
 3   card_type              6146 non-null   object        
 4   card_number            6146 non-null   int64         
 5   expires                6146 non-null   datetime64[ns]
 6   cvv                    6146 non-null   int64         
 7   has_chip               6146 non-null   object        
 8   num_cards_issued       6146 non-null   int64         
 9   credit_limit           6146 non-null   int64         
 10  acct_open_date         6146 non-null   datetime64[ns]
 11  year_pin_last_changed  6146 non-null   int64         
 12  card_on_dark_web       6146 non-null   object        
dtypes: 

In [42]:
df_cards.loc[df_cards['credit_limit'] < 0]

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web


In [43]:
df_cards.loc[df_cards['acct_open_date'] > df_cards['expires']]

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web


In [44]:
today = pd.Timestamp.today().normalize()
df_cards.loc[df_cards['expires'] > today]

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web


In [45]:
df_cards['year_pin_last_changed'].unique()

array([2008, 2014, 2004, 2012, 2009, 2011, 2015, 2019, 2007, 2010, 2020,
       2017, 2016, 2013, 2018, 2006, 2005, 2003, 2002])

In [46]:
df_cards.loc[df_cards['acct_open_date'].dt.year > df_cards['year_pin_last_changed']]

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web


In [47]:
df_cards['card_on_dark_web'].unique()

array(['No'], dtype=object)

# **USERS DATA**

In [50]:
df_users = pd.read_csv('/kaggle/input/transactions-fraud-datasets/users_data.csv')
df_users.head(5)

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1


In [51]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 2000 non-null   int64  
 1   current_age        2000 non-null   int64  
 2   retirement_age     2000 non-null   int64  
 3   birth_year         2000 non-null   int64  
 4   birth_month        2000 non-null   int64  
 5   gender             2000 non-null   object 
 6   address            2000 non-null   object 
 7   latitude           2000 non-null   float64
 8   longitude          2000 non-null   float64
 9   per_capita_income  2000 non-null   object 
 10  yearly_income      2000 non-null   object 
 11  total_debt         2000 non-null   object 
 12  credit_score       2000 non-null   int64  
 13  num_credit_cards   2000 non-null   int64  
dtypes: float64(2), int64(7), object(5)
memory usage: 218.9+ KB


In [52]:
df_users['id'].duplicated().sum()

0

In [53]:
df_users['current_age'].unique()

array([ 53,  81,  63,  43,  42,  36,  26,  34,  27,  29,  18,  48,  41,
        54,  76,  22,  66,  28,  70,  45,  40,  37,  49,  33,  21,  47,
        51,  46,  59,  91,  79,  57,  60,  61,  55,  44,  82,  90,  30,
        23,  65,  87,  58,  25,  32,  64,  52,  75,  62,  56,  35,  24,
        19,  20,  39,  86,  67,  83,  68,  92,  84,  38,  80,  31,  50,
        77,  88, 101,  74,  73,  85,  78,  72,  69,  71,  89,  98,  93,
        94,  99])

In [54]:
df_users['age_group'] = pd.cut(df_users['current_age'], 
                            bins=[0, 25, 35, 50, 65, 101], 
                            labels=['18-25', '26-35', '36-50', '51-65', '65+'])
df_users.head(5)

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,age_group
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5,51-65
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5,51-65
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5,65+
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4,51-65
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1,36-50


In [55]:
df_users['credit_score_level'] = pd.cut(df_users['credit_score'],
                                bins = [0, 579, 669, 739, 799, 850],
                                labels = ["Very High Risk", "High Risk", "Medium Risk", "Low Risk", "Very Low Risk"])
df_users.head(5)

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,age_group,credit_score_level
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5,51-65,Low Risk
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5,51-65,Medium Risk
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5,65+,Medium Risk
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4,51-65,Medium Risk
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1,36-50,Medium Risk


In [56]:
def clean_amount(col):
    return pd.to_numeric(col.astype(str).replace('[\$,]', '', regex=True), errors='coerce')
    
df_users['per_capita_income'] = clean_amount(df_users['per_capita_income'])
df_users['yearly_income'] = clean_amount(df_users['yearly_income'])
df_users['total_debt'] = clean_amount(df_users['total_debt'])

df_users.head(5)

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,age_group,credit_score_level
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278,59696,127613,787,5,51-65,Low Risk
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,37891,77254,191349,701,5,51-65,Medium Risk
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,22681,33483,196,698,5,65+,Medium Risk
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,163145,249925,202328,722,4,51-65,Medium Risk
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,53797,109687,183855,675,1,36-50,Medium Risk


In [57]:
df_users['total_debt'].describe()

count      2000.000000
mean      63709.694000
std       52254.453421
min           0.000000
25%       23986.750000
50%       58251.000000
75%       89070.500000
max      516263.000000
Name: total_debt, dtype: float64

In [58]:
df_users['debt_level'] = pd.cut(df_users['total_debt'],
                                bins = [0, 25000, 50000, 100000, 250000,600000],
                                labels = ['Very Low Debt', 'Low Debt', 'Medium Debt', 'High Debt', 'Very High Debt'],
                                include_lowest=True,
                                right=True )

df_users.head(5)

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,age_group,credit_score_level,debt_level
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278,59696,127613,787,5,51-65,Low Risk,High Debt
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,37891,77254,191349,701,5,51-65,Medium Risk,High Debt
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,22681,33483,196,698,5,65+,Medium Risk,Very Low Debt
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,163145,249925,202328,722,4,51-65,Medium Risk,High Debt
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,53797,109687,183855,675,1,36-50,Medium Risk,High Debt


In [60]:
df_users['retirement_age'].unique()

array([66, 68, 67, 63, 70, 60, 62, 71, 57, 65, 64, 58, 72, 69, 55, 75, 52,
       59, 73, 56, 61, 74, 54, 78, 79, 53, 76, 50, 77])

In [61]:
df_users['birth_month'].unique()

array([11, 12,  1,  9, 10,  7,  6,  5,  4,  8,  2,  3])

In [62]:
df_users['gender'].unique()

array(['Female', 'Male'], dtype=object)

In [63]:
df_users['yearly_income'].describe()

count      2000.000000
mean      45715.882000
std       22992.615456
min           1.000000
25%       32818.500000
50%       40744.500000
75%       52698.500000
max      307018.000000
Name: yearly_income, dtype: float64

In [64]:
df_users['income_range'] = pd.cut(df_users['yearly_income'], 
                                 bins=[0, 30000, 60000, 100000, 200000, df_users['yearly_income'].max()], 
                                 labels=['Low', 'Lower-Mid', 'Mid', 'Upper-Mid', 'High'])
df_users.head(5)

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,age_group,credit_score_level,debt_level,income_range
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278,59696,127613,787,5,51-65,Low Risk,High Debt,Lower-Mid
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,37891,77254,191349,701,5,51-65,Medium Risk,High Debt,Mid
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,22681,33483,196,698,5,65+,Medium Risk,Very Low Debt,Lower-Mid
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,163145,249925,202328,722,4,51-65,Medium Risk,High Debt,High
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,53797,109687,183855,675,1,36-50,Medium Risk,High Debt,Upper-Mid


In [65]:
df_users['credit_score'].describe()

count    2000.000000
mean      709.734500
std        67.221949
min       480.000000
25%       681.000000
50%       711.500000
75%       753.000000
max       850.000000
Name: credit_score, dtype: float64

In [66]:
df_users['num_credit_cards'].unique()

array([5, 4, 1, 3, 2, 6, 7, 8, 9])

## **MCC CODE DESCRIPTION**

In [62]:
mcc_code = pd.read_json('/kaggle/input/transactions-fraud-datasets/mcc_codes.json', orient='index')
mcc_code.reset_index(inplace=True)
mcc_code.columns = ['mcc_id', 'catogory']
mcc_code.head(5)

,mcc_id,catogory
0,5812,Eating Places and Restaurants
1,5541,Service Stations
2,7996,"Amusement Parks, Carnivals, Circuses"
3,5411,"Grocery Stores, Supermarkets"
4,4784,Tolls and Bridge Fees


In [63]:
mcc_code['mcc_id'].duplicated().sum()

0

In [3]:
fraud_labels['transaction_id'].duplicated().sum()

0

In [5]:
fraud_labels['is_fraud'].unique()

array(['No', 'Yes'], dtype=object)